In [ ]:
# Recurrent Neural Networks

Neural networks are another powerful way of learning functions.

Recurrent neural networks (RNNs) are particularly useful for applications involve _sequences_ of data. Data that comes in meaningful sequences, like natural language (a sequence of words), typically means that the context of any data point (i.e. the history that leads up to it) is just as relevant as the data point itself. The neural network must have some kind of "memory"; this is what RNNs essentially do (though their memory is very short-term).

So we can use RNNs to learn something like a language model, which can in some sense learn the style of a particular author and riff off of an input sequence of words (e.g. a sentence) in that style. There are examples abound here, such as generated texts in the style of [Finnegan's Wake](https://www.countbayesie.com/blog/2015/5/24/writing-finnegans-wake-with-a-recurrent-neural-net), endless new episode scripts for [Full House](http://fullest.house/), and algorithimically-composed [sheet music](https://medium.com/dbrs-innovation-labs/in-his-novel-galatea-2-2-e9d11c9b7c2a#.xd00cremz).

So pretty much anything that can be thought of as a sequence can be handled by an RNN. Ultimately we are still just learning a function that describes the dynamics in whatever sequence we're looking at, it's just more complex.

In [1]:
import numpy as np
import pandas as pd

In [9]:
df_ = pd.read_csv('events.csv')

In [10]:
df_.head(100)

,Event ID,Event Date,Source Name,Source Sectors,Source Country,Event Text,CAMEO Code,Intensity,Target Name,Target Sectors,Target Country,Story ID,Sentence Number,Publisher,City,District,Province,Country,Latitude,Longitude
0,20718170,2014-01-01,Police (Australia),"Police,Government",Australia,"Arrest, detain, or charge with legal action",173,-5.0,Men (Australia),"Social,General Population / Civilian / Social",Australia,32493690,2,Daily Telegraph,Surfers Paradise,Gold Coast,State of Queensland,Australia,-28.00270,153.4300
1,20718171,2014-01-01,Police (Australia),"Police,Government",Australia,"Arrest, detain, or charge with legal action",173,-5.0,Children (Australia),"Social,General Population / Civilian / Social",Australia,32493693,1,Daily Telegraph,Maroubra,Randwick,State of New South Wales,Australia,-33.95000,151.2330
2,20718172,2014-01-01,Government Official (Democratic Republic of Co...,Government,Democratic Republic of Congo,Make statement,10,0.0,Attacker (Democratic Republic of Congo),"Criminals / Gangs,Dissident",Democratic Republic of Congo,32495112,3,The Australian,Kinshasa,NaN,Kinshasa City,Democratic Republic of Congo,-4.32142,15.3081
3,20718174,2014-01-01,Military (South Sudan),"Military,Government",South Sudan,Use conventional military force,190,-10.0,Armed Rebel (South Sudan),"Rebel,Dissident",South Sudan,32495113,1,The Australian,Juba,NaN,Central Equatoria State,South Sudan,4.85165,31.5825
4,20718173,2014-01-01,Armed Rebel (South Sudan),"Rebel,Dissident",South Sudan,Use unconventional violence,180,-9.0,Military (South Sudan),"Military,Government",South Sudan,32495113,1,The Australian,Juba,NaN,Central Equatoria State,South Sudan,4.85165,31.5825
5,20718176,2014-01-01,Military (South Sudan),"Military,Government",South Sudan,Use conventional military force,190,-10.0,Rebel Group (South Sudan),"Rebel,Dissident",South Sudan,32495113,4,The Australian,Juba,NaN,Central Equatoria State,South Sudan,4.85165,31.5825
6,20718175,2014-01-01,Rebel Group (South Sudan),"Rebel,Dissident",South Sudan,Use unconventional violence,180,-9.0,Military (South Sudan),"Military,Government",South Sudan,32495113,4,The Australian,Juba,NaN,Central Equatoria State,South Sudan,4.85165,31.5825
7,20718178,2014-01-01,Police (Australia),"Police,Government",Australia,fight with small arms and light weapons,193,-10.0,Criminal (Australia),"Dissident,Criminals / Gangs",Australia,32495121,1,The Australian,NaN,NaN,State of Victoria,Australia,-37.00000,145.0000
8,20718179,2014-01-01,Police (Australia),"Police,Government",Australia,Investigate,90,-2.0,Citizen (Australia),"Social,General Population / Civilian / Social",Australia,32495125,5,The Australian,NaN,NaN,NaN,Australia,-35.28350,149.1280
9,20718180,2014-01-01,Police (Australia),"Police,Government",Australia,"Arrest, detain, or charge with legal action",173,-5.0,Women (Australia),"Social,General Population / Civilian / Social",Australia,32495126,3,The Australian,Leichhardt,Leichhardt,State of New South Wales,Australia,-33.88330,151.1500


In [27]:
len(df_)

120000

In [5]:
# Source and Target Sectors are kind of redundant with Source and Target Names
cols = ['Source Name', 'Source Country', 'Event Text', 'Target Name', 'Target Country']
df = df_.loc[:, cols]

# the category label space is massive, this reduces it
df['Source Name'].replace(to_replace='\s\(.+', value='', inplace=True, regex=True)
df['Target Name'].replace(to_replace='\s\(.+', value='', inplace=True, regex=True) 

# NaN values are when source/target don't have a country
# df[df.isnull().any(axis=1)]
df.fillna('NONE', inplace=True)

df.head()

,Source Name,Source Country,Event Text,Target Name,Target Country
0,Police,Australia,"Arrest, detain, or charge with legal action",Men,Australia
1,Police,Australia,"Arrest, detain, or charge with legal action",Children,Australia
2,Government Official,Democratic Republic of Congo,Make statement,Attacker,Democratic Republic of Congo
3,Military,South Sudan,Use conventional military force,Armed Rebel,South Sudan
4,Armed Rebel,South Sudan,Use unconventional violence,Military,South Sudan


In [6]:
cat_map = {}
for col in cols:
    cat = df.loc[:,col].astype('category')
    cat_map[col] = cat.cat.categories
    df.loc[:,col] = cat.cat.codes

In [7]:
cat_map['Source Country'][df['Source Country'][0]]

'Australia'

In [10]:
n_samples = len(df)
data = df.as_matrix()
data

NameError: name 'df' is not defined

In [9]:
Y = data[1:]
Y.shape

(119999, 5)

In [10]:
data = data[:-1]
data.shape

(119999, 5)

In [11]:
vocab_size = np.max(data) + 1
vocab_size

11132

In [12]:
# check how many GB of memory will be needed
# using int16
print(Y.shape[0] * Y.shape[1] * (Y.max() + 1) * 16 * 1.25e-10)

y_onehot = np.zeros((Y.shape[0], Y.shape[1], Y.max() + 1))
layer_idx = np.arange(Y.shape[0]).reshape(Y.shape[0], 1)
component_idx = np.tile(np.arange(Y.shape[1]), (Y.shape[0], 1))
y_onehot[layer_idx, component_idx, Y] = 1

13.35828868


In [13]:
from keras.layers.embeddings import Embedding
from keras.layers.core import Dropout, Dense
from keras.layers.recurrent import LSTM
from keras.models import Sequential
from keras.layers.wrappers import TimeDistributed

embed_size = 100
batch_size = 128
epochs = 100

model = Sequential()
model.add(Embedding(vocab_size, embed_size, input_length=data.shape[1]))
model.add(Dropout(0.3))
model.add(LSTM(embed_size, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(embed_size, return_sequences=True))
model.add(Dropout(0.3))
model.add(TimeDistributed(Dense(vocab_size, activation='softmax')))

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(data, y_onehot, batch_size=batch_size, nb_epoch=epochs, validation_split=0.05)

Train on 113999 samples, validate on 6000 samples
Epoch 1/100
113999/113999 [==============================] - 92s - loss: 5.7559 - acc: 0.0742 - val_loss: 5.1997 - val_acc: 0.1028
Epoch 2/100
113999/113999 [==============================] - 86s - loss: 5.1029 - acc: 0.1126 - val_loss: 4.9846 - val_acc: 0.1141
Epoch 3/100
113999/113999 [==============================] - 85s - loss: 4.9439 - acc: 0.1238 - val_loss: 4.8367 - val_acc: 0.1219
Epoch 4/100
113999/113999 [==============================] - 85s - loss: 4.8147 - acc: 0.1372 - val_loss: 4.6941 - val_acc: 0.1475
Epoch 5/100
113999/113999 [==============================] - 86s - loss: 4.6821 - acc: 0.1561 - val_loss: 4.5677 - val_acc: 0.1680
Epoch 6/100
113999/113999 [==============================] - 85s - loss: 4.5602 - acc: 0.1729 - val_loss: 4.4449 - val_acc: 0.1956
Epoch 7/100
113999/113999 [==============================] - 84s - loss: 4.4480 - acc: 0.1897 - val_loss: 4.3503 - val_acc: 0.2107
Epoch 8/100
113999/113999 [======

Using TensorFlow backend.


In [14]:
model.save_weights('icews_weights.h5', overwrite=True)

[WARNING] icews_weights.h5 already exists - overwrite? [y/n]y
[TIP] Next time specify overwrite=True in save_weights!


In [15]:
def render(arr):
    parts = []
    for idx, col in enumerate(cols):
        parts.append(cat_map[col][int(arr[idx])])
    return parts

In [39]:
def ramble(seed, n):
    rambling = [seed]
    for _ in range(n):
        probs = model.predict(seed.T)[0]
        seed = []
        # random draw based on probs
        cs = np.cumsum(probs, axis=1)
        for row in cs:          
            idx = row.searchsorted(np.random.random() * row[-1], 'right')
            seed.append(idx)
        rambling.append(render(seed))
        seed = np.array(seed).reshape(5,1)
    return rambling

In [44]:
seed = np.random.randint(0, 20, (5,1))
for source_name, source_country, event_text, target_name, target_country in ramble(seed, 10):
    print('{} ({}) -> {} -> {} ({})'.format(source_name, source_country, event_text, target_name, target_country))

Citizen (Turkey) -> Protest violently, riot -> Argentina (Italy)
Communist Party of India (Turkey) -> Refuse to release persons or property -> Activist (Indonesia)
High Ranking Military Personnel (NONE) -> Make statement -> Militant (Switzerland)
Domestic Affairs (South Korea) -> Accuse -> Indonesia (Turkey)
Media (South Korea) -> Appeal for judicial cooperation -> Japan (India)
Protester (United Kingdom) -> Consult -> Men (United States)
Citizen (Australia) -> Consult -> Protester (Thailand)
Barack Obama (Thailand) -> Consult -> Men (South Korea)
Barack Obama (United States) -> Consult -> Unspecified Actor (United States)
Barack Obama (France) -> Cooperate economically -> Lakhdar Brahimi (Saudi Arabia)
